In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer, TextStreamer
import pandas as pd

# Load Quantized Model w/ PEFT

In [ ]:
model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Load Dataset + Splitting (Train,Val)

# Tokenize Dataset

In [ ]:
train_encodings = tokenizer(train_dataset, truncation=True, padding=True, max_length=300, return_tensors='pt')
val_encodings = tokenizer(val_dataset, truncation=True, padding=True, max_length=300, return_tensors='pt')

# Convert to PyTorch

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = TextDataset(train_encodings)
val_dataset = TextDataset(val_encodings)

# Define A Generate Function

In [ ]:
def generate(input):
    encoding = tokenizer(input, return_tensors="pt").to("cuda:0")
    streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens= True)
    model.generate(input_ids=encoding.input_ids, streamer = streamer,attention_mask=encoding.attention_mask, max_new_tokens=4096, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)


# Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=10,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# Save the LoRA

In [ ]:
model.save_pretrained("fine-tuned-llama7b-10epochs")